In [1]:
## https://www.kaggle.com/kanncaa1/recommendation-systems-tutorial/data
## https://www.kaggle.com/gspmoreira/recommender-systems-in-python-101

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import scipy as scipy
import math as math
import random as random
import sklearn

from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds

%matplotlib inline

In [3]:
movies_df = pd.read_csv('movie.csv')

In [4]:
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
movies_df['title_1'] = movies_df['title'].str.split("\(\d", expand=True)[0]
movies_df['year'] = movies_df['title'].str.extract('.*\((.*\d{4})\).*', expand=True)
movies_df.head(30)

,movieId,title,genres,title_1,year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Toy Story,1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,Jumanji,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,Grumpier Old Men,1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,Waiting to Exhale,1995
4,5,Father of the Bride Part II (1995),Comedy,Father of the Bride Part II,1995
5,6,Heat (1995),Action|Crime|Thriller,Heat,1995
6,7,Sabrina (1995),Comedy|Romance,Sabrina,1995
7,8,Tom and Huck (1995),Adventure|Children,Tom and Huck,1995
8,9,Sudden Death (1995),Action,Sudden Death,1995
9,10,GoldenEye (1995),Action|Adventure|Thriller,GoldenEye,1995


In [6]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27278 entries, 0 to 27277
Data columns (total 5 columns):
movieId    27278 non-null int64
title      27278 non-null object
genres     27278 non-null object
title_1    27278 non-null object
year       27257 non-null object
dtypes: int64(1), object(4)
memory usage: 1.0+ MB


In [7]:
interactions_df = pd.read_csv('rating_1.csv')
interactions_df.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,4/2/2005 23:53
1,1,29,3.5,4/2/2005 23:31
2,1,32,3.5,4/2/2005 23:33
3,1,47,3.5,4/2/2005 23:32
4,1,50,3.5,4/2/2005 23:29


In [8]:
interactions_df.drop('timestamp', axis=1, inplace=True)

In [9]:
interactions_df.head()

,userId,movieId,rating
0,1,2,3.5
1,1,29,3.5
2,1,32,3.5
3,1,47,3.5
4,1,50,3.5


In [10]:
interactions_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 3 columns):
userId     20000 non-null int64
movieId    20000 non-null int64
rating     20000 non-null float64
dtypes: float64(1), int64(2)
memory usage: 468.8 KB


In [11]:
interactions_df = interactions_df.iloc[:20000,:]

In [12]:
interactions_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 3 columns):
userId     20000 non-null int64
movieId    20000 non-null int64
rating     20000 non-null float64
dtypes: float64(1), int64(2)
memory usage: 468.8 KB


In [13]:
print(interactions_df['userId'].nunique())
print(interactions_df['movieId'].nunique())

156
4192


In [14]:
## Merge movies_df (with only movieId, title columns) with interactions_df

interactions_movies_df = interactions_df.merge(movies_df[['movieId', 'title_1', 'year', 'genres']], how = 'left', left_on = 'movieId',
               right_on = 'movieId')

In [15]:
interactions_movies_df.head(30)

,userId,movieId,rating,title_1,year,genres
0,1,2,3.5,Jumanji,1995,Adventure|Children|Fantasy
1,1,29,3.5,"City of Lost Children, The (Cité des enfants p...",1995,Adventure|Drama|Fantasy|Mystery|Sci-Fi
2,1,32,3.5,Twelve Monkeys (a.k.a. 12 Monkeys),1995,Mystery|Sci-Fi|Thriller
3,1,47,3.5,Seven (a.k.a. Se7en),1995,Mystery|Thriller
4,1,50,3.5,"Usual Suspects, The",1995,Crime|Mystery|Thriller
5,1,112,3.5,Rumble in the Bronx (Hont faan kui),1995,Action|Adventure|Comedy|Crime
6,1,151,4.0,Rob Roy,1995,Action|Drama|Romance|War
7,1,223,4.0,Clerks,1994,Comedy
8,1,253,4.0,Interview with the Vampire: The Vampire Chroni...,1994,Drama|Horror
9,1,260,4.0,Star Wars: Episode IV - A New Hope,1977,Action|Adventure|Sci-Fi


In [16]:
interactions_movies_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20000 entries, 0 to 19999
Data columns (total 6 columns):
userId     20000 non-null int64
movieId    20000 non-null int64
rating     20000 non-null float64
title_1    20000 non-null object
year       20000 non-null object
genres     20000 non-null object
dtypes: float64(1), int64(2), object(3)
memory usage: 1.1+ MB


## Popularity Model

In [17]:
#interactions_movies_df[interactions_movies_df['movieId']==1][['rating']].mean().reset_index()

interactions_movies_grp = interactions_movies_df.groupby(['movieId', 'title_1'])[['rating']].sum().reset_index()
interactions_movies_grp.rename(columns = {'rating': 'TotalRating'},inplace=True)
interactions_movies_grp.head()

,movieId,title_1,TotalRating
0,1,Toy Story,221.5
1,2,Jumanji,60.0
2,3,Grumpier Old Men,59.5
3,4,Waiting to Exhale,8.0
4,5,Father of the Bride Part II,35.5


In [18]:
#Sort on desc order of TotalRating

popular_movies_df = interactions_movies_grp.sort_values(by='TotalRating', ascending=False).reset_index(drop=True)
popular_movies_df.head(10)

,movieId,title_1,TotalRating
0,296,Pulp Fiction,339.5
1,356,Forrest Gump,324.0
2,318,"Shawshank Redemption, The",312.5
3,593,"Silence of the Lambs, The",273.0
4,480,Jurassic Park,266.5
5,260,Star Wars: Episode IV - A New Hope,231.0
6,50,"Usual Suspects, The",228.5
7,608,Fargo,228.5
8,2858,American Beauty,223.0
9,1,Toy Story,221.5


In [19]:
class PopularityRecommender:
    
    MODEL_NAME = 'Popularity'
    
    def __init__(self, popularity_df):
        self.popularity_df = popularity_df
        
    def get_model_name(self):
        return self.MODEL_NAME            
    
    def recommend_items(self, userid, already_watched=[], topn=10):
        popular_recommendations_df = self.popularity_df[~self.popularity_df['movieId'].isin(already_watched)].reset_index(drop=True).head(topn)
        popular_recommendations_df['Rank'] = popular_recommendations_df['TotalRating'].rank(ascending=0, method='first')
        return popular_recommendations_df
    

### Instantiate the PopularityRecommender class

In [20]:
popular_rec_model = PopularityRecommender(popular_movies_df)

In [21]:
### Split train and test set

train_df, test_df = train_test_split(interactions_movies_df, stratify=interactions_movies_df['userId'], test_size=0.2, random_state=42)

In [22]:
print(train_df.shape, test_df.shape)

(16000, 6) (4000, 6)


In [23]:
train_df['userId'].nunique()

156

### Call recommend_items for user in train df

In [26]:
users_train = train_df['userId'].tolist()
train_userid = users_train[1]

## movies already watched by train_userid
already_watched_train = train_df[train_df['userId']==train_userid]['movieId'].tolist()


print("Popular movie recommendations for user in train_df: ", train_userid)
popular_rec_model.recommend_items(train_userid, already_watched_train)

Popular movie recommendations for user in train_df:  131


,movieId,title_1,TotalRating,Rank
0,318,"Shawshank Redemption, The",312.5,1.0
1,593,"Silence of the Lambs, The",273.0,2.0
2,260,Star Wars: Episode IV - A New Hope,231.0,3.0
3,50,"Usual Suspects, The",228.5,4.0
4,608,Fargo,228.5,5.0
5,47,Seven (a.k.a. Se7en),217.0,6.0
6,589,Terminator 2: Judgment Day,216.5,7.0
7,527,Schindler's List,210.5,8.0
8,590,Dances with Wolves,200.5,9.0
9,457,"Fugitive, The",199.5,10.0


In [27]:
print("Movies already watched by user in train_df: ", train_userid)
train_df[train_df['userId']==train_userid][['movieId', 'year', 'rating', 'title_1', 'genres']].sort_values(by='year', ascending=False).head(5)

Movies already watched by user in train_df:  131


,movieId,year,rating,title_1,genres
16125,67788,2009,3.0,Confessions of a Shopaholic,Comedy|Romance
16130,69644,2009,2.5,Ice Age: Dawn of the Dinosaurs,Action|Adventure|Animation|Children|Comedy|Rom...
16131,71264,2009,3.0,Cloudy with a Chance of Meatballs,Animation|Children|Fantasy|IMAX
16128,69406,2009,1.0,"Proposal, The",Comedy|Romance
16122,65585,2009,1.5,Bride Wars,Comedy|Romance


### Call recommend_items for SAME user in test df

In [28]:
#users_test = test_df['userId'].tolist()
#test_userid = 131 #users_test[1]

## movies already watched by train_userid
already_watched_test = test_df[test_df['userId']==train_userid]['movieId'].tolist()


print("Popular movie recommendations for user in test_df: ", train_userid)
popular_rec_model.recommend_items(train_userid, already_watched_test)

Popular movie recommendations for user in test_df:  131


,movieId,title_1,TotalRating,Rank
0,296,Pulp Fiction,339.5,1.0
1,356,Forrest Gump,324.0,2.0
2,318,"Shawshank Redemption, The",312.5,3.0
3,480,Jurassic Park,266.5,4.0
4,260,Star Wars: Episode IV - A New Hope,231.0,5.0
5,50,"Usual Suspects, The",228.5,6.0
6,608,Fargo,228.5,7.0
7,2858,American Beauty,223.0,8.0
8,1,Toy Story,221.5,9.0
9,589,Terminator 2: Judgment Day,216.5,10.0


In [30]:
print("Movies already watched by user in test_df: ", train_userid)
test_df[test_df['userId']==train_userid][['movieId', 'year', 'rating', 'title_1', 'genres']].sort_values(by='year', ascending=False).head(5)

Movies already watched by user in test_df:  131


,movieId,year,rating,title_1,genres
16112,62265,2009,0.5,"Accidental Husband, The",Comedy|Romance
16127,69122,2009,2.0,"Hangover, The",Comedy|Crime
16129,69436,2009,2.5,Year One,Adventure|Comedy
16126,68954,2009,2.5,Up,Adventure|Animation|Children|Drama
16120,64032,2008,0.5,Four Christmases,Comedy


In [37]:
print("Total number of movies watched by user %d in test_df is %d " %(train_userid, test_df[test_df['userId']==train_userid].shape[0]))

Total number of movies watched by user 131 in test_df is 86 


In [38]:
#TP = Predicted movies that user likes

#TP + FN = Movies that are actually watched by user
# recall = TP / (TP+FN)

In [39]:
## 1. Get list of items that a user has interacted in interactions full data set(train+test)
## 2. Get the recommended items based on model (Popularity, content-based, collab-filtering)
## 3 Get the items intereacted in test set
## 3.1A For evey item in the interacted_test_set
## 3.1B Filter the items that were recommended (#3) 
## 3.1C If the item in the interacted_test_set is in top 5 or top10 of the recommended_list, count as hits_5 or hits_10 resply.
## 4. Calculate recall_5 as hits_5/total count of interacted_test_set and recall_10 = hits_10/total count of interacted_test_set

In [51]:
class ModelEvaluator:
    
    def already_watched(self, df, userid):
        already_watched_items = set(df[df['userId']==userid]['movieId'])
        return already_watched_items
    
    def evaluate_model_for_user(self, model, userid):
        user_metrics = {}
        
        already_watched_train = self.already_watched(train_df, userid)
        rec_df = model.recommend_items(userid, already_watched_train, topn=20)
        rec_items = rec_df['movieId'].values
        
        already_watched_test = self.already_watched(test_df, userid)
        hits_5 = 0
        hits_10 = 0
             
        for test_movie_watched in set(already_watched_test):
            for idx, rec_movie_id in enumerate(rec_items):
                try:
                    if(test_movie_watched == rec_movie_id):
                        index = idx
                        if(index < 5):
                            hits_5 += 1
                        if(index < 10):
                            hits_10 += 1
                except:
                    index = -1
         
        recall_5 = (hits_5 / len(already_watched_test))*100
        recall_10 = (hits_10 / len(already_watched_test))*100
        
        recall_5 = "{:.2f}".format(recall_5)
        recall_10 = "{:.2f}".format(recall_10)
                        
        user_metrics = {#'hits@05_count':hits_5, 
                      'hits@10_count':hits_10, 
                      'test_interacted_count': len(already_watched_test),
                      #'recall@05': recall_5,
                      'recall@10': recall_10
                       }
        
        return user_metrics
        
    def evaluate_model(self, model):
            print('Running evaluation for users')
            metrics = []
            for idx, user_id in enumerate(list(test_df['userId'].unique())):                  
                user_metrics = self.evaluate_model_for_user(model, user_id)  
                user_metrics['_person_id'] = user_id
                metrics.append(user_metrics)
                
            print('Number of users processed: ', (idx+1))        

            detailed_results_df = pd.DataFrame(metrics).sort_values('test_interacted_count', ascending=False).reset_index(drop=True)
        
            #global_recall_at_5 = (detailed_results_df['hits@05_count'].sum() / float(detailed_results_df['test_interacted_count'].sum()))*100
            global_recall_at_10 = (detailed_results_df['hits@10_count'].sum() / float(detailed_results_df['test_interacted_count'].sum()))*100
        
            #global_recall_at_5 = "{:.2f}".format(global_recall_at_5)
            global_recall_at_10 = "{:.2f}".format(global_recall_at_10)
            
            global_metrics = {'modelName': model.get_model_name(),
                              #'recall@05': global_recall_at_5,
                              'recall@10': global_recall_at_10}    
       
            return global_metrics, detailed_results_df   
    

In [52]:
model_eval = ModelEvaluator()

In [53]:
print('Evaluating Popularity recommendation model...')
pop_global_metrics, pop_detailed_results_df = model_eval.evaluate_model(popular_rec_model)

Evaluating Popularity recommendation model...
Running evaluation for users
Number of users processed:  156


In [54]:
pop_global_metrics

{'modelName': 'Popularity', 'recall@10': '5.88'}

In [55]:
pop_detailed_results_df.head()

,_person_id,hits@10_count,recall@10,test_interacted_count
0,116,7,3.15,222
1,104,4,2.00,200
2,54,7,4.93,142
3,91,10,7.52,133
4,58,3,2.29,131


In [66]:
#pop_detailed_results_df[pop_detailed_results_df['_person_id'] == 131]

In [68]:
#already_watched_131 = train_df[train_df['userId']==131]['movieId'].tolist()
#rec_df_131 = popular_rec_model.recommend_items(131, already_watched_131)
#test_interacted_movies_131 = set(test_df[test_df['userId']==131]['movieId'])
#len(test_interacted_movies_131)

#hits = 0
#for item_id in test_interacted_movies_131: ## 41 movies
#    valid_recs = rec_df_131['movieId'].values
    
#    for i, m in enumerate(valid_recs):
#        try:
#            if(m == item_id):
#                index = i
##                print(i, m, item_id)
#                if(index < 10):
#                    hits += 1
#        except:
#            index = -1
#print(hits, len(test_interacted_movies_131), hits/len(test_interacted_movies_131))        

## Content based Filtering model

In [69]:
## Content-based filtering approaches leverage description or attributes from items the user has interacted 
## to recommend similar items.
## It depends only on the user's previous choices, making this method robust to avoid the cold-start problem. 

In [70]:
#Ignoring stopwords (words with no semantics) from English
stopwords_list = stopwords.words('english')

#Trains a model whose vectors size is 5000, composed by the main unigrams and bigrams found in the corpus, ignoring stopwords
vectorizer = TfidfVectorizer(analyzer='word',
                     ngram_range= (1,2),
                     min_df=0.003,
                     max_df=0.5,
                     max_features=5000,
                     stop_words=stopwords_list)

In [73]:
#title_genre = pd.concat([movies_df['Title'], movies_df['Genre']])
#title_genre = movies_df['Title'] + " " + movies_df['Genre']
genres = movies_df['genres'].tolist()

In [75]:
#genres

In [76]:
tfidf_matrix = vectorizer.fit_transform(genres)

In [77]:
tfidf_feature_names = vectorizer.get_feature_names()

In [78]:
print(len(tfidf_feature_names))
print(tfidf_feature_names)

90
['action', 'action adventure', 'action animation', 'action comedy', 'action crime', 'action drama', 'action horror', 'action sci', 'action thriller', 'adventure', 'adventure animation', 'adventure children', 'adventure comedy', 'adventure crime', 'adventure drama', 'adventure fantasy', 'adventure sci', 'animation', 'animation children', 'animation comedy', 'children', 'children comedy', 'children drama', 'children fantasy', 'comedy', 'comedy crime', 'comedy documentary', 'comedy drama', 'comedy fantasy', 'comedy horror', 'comedy musical', 'comedy romance', 'comedy sci', 'comedy western', 'crime', 'crime drama', 'crime horror', 'crime mystery', 'crime thriller', 'documentary', 'documentary drama', 'documentary musical', 'drama', 'drama fantasy', 'drama film', 'drama horror', 'drama musical', 'drama mystery', 'drama romance', 'drama sci', 'drama thriller', 'drama war', 'drama western', 'fantasy', 'fantasy horror', 'fantasy musical', 'fantasy mystery', 'fantasy romance', 'fantasy sci',

In [79]:
item_ids = movies_df['movieId'].tolist()
#item_ids.index(-8949113594875411859)
#item_ids

In [80]:
interactions_movies_df.head()

,userId,movieId,rating,title_1,year,genres
0,1,2,3.5,Jumanji,1995,Adventure|Children|Fantasy
1,1,29,3.5,"City of Lost Children, The (Cité des enfants p...",1995,Adventure|Drama|Fantasy|Mystery|Sci-Fi
2,1,32,3.5,Twelve Monkeys (a.k.a. 12 Monkeys),1995,Mystery|Sci-Fi|Thriller
3,1,47,3.5,Seven (a.k.a. Se7en),1995,Mystery|Thriller
4,1,50,3.5,"Usual Suspects, The",1995,Crime|Mystery|Thriller


In [81]:
users_movies_df = interactions_movies_df.set_index('userId')

In [82]:
users_movies_df.head()

,movieId,rating,title_1,year,genres
userId,,,,,
1,2,3.5,Jumanji,1995,Adventure|Children|Fantasy
1,29,3.5,"City of Lost Children, The (Cité des enfants p...",1995,Adventure|Drama|Fantasy|Mystery|Sci-Fi
1,32,3.5,Twelve Monkeys (a.k.a. 12 Monkeys),1995,Mystery|Sci-Fi|Thriller
1,47,3.5,Seven (a.k.a. Se7en),1995,Mystery|Thriller
1,50,3.5,"Usual Suspects, The",1995,Crime|Mystery|Thriller


In [83]:
users_movies_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20000 entries, 1 to 156
Data columns (total 5 columns):
movieId    20000 non-null int64
rating     20000 non-null float64
title_1    20000 non-null object
year       20000 non-null object
genres     20000 non-null object
dtypes: float64(1), int64(1), object(3)
memory usage: 937.5+ KB


In [84]:
users_movies_df.dropna(inplace=True)

In [85]:
users_movies_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20000 entries, 1 to 156
Data columns (total 5 columns):
movieId    20000 non-null int64
rating     20000 non-null float64
title_1    20000 non-null object
year       20000 non-null object
genres     20000 non-null object
dtypes: float64(1), int64(1), object(3)
memory usage: 937.5+ KB


#### To model the user profile, we take all the item profiles the user has interacted with and average them.

#### The average is weighted by the rating

In [86]:
users_movies_df.groupby([users_movies_df.index])[['movieId']].count().sort_values(by='movieId', ascending=True).head(5)

,movieId
userId,
36,20
52,20
39,20
37,20
123,22


In [87]:
users_movies_df.loc[36].sort_values(by='rating', ascending=False)

,movieId,rating,title_1,year,genres
userId,,,,,
36,2353,4.0,Enemy of the State,1998,Action|Thriller
36,145,3.5,Bad Boys,1995,Action|Comedy|Crime|Drama|Thriller
36,1358,3.5,Sling Blade,1996,Drama
36,2023,3.5,"Godfather: Part III, The",1990,Crime|Drama|Mystery|Thriller
36,1408,3.5,"Last of the Mohicans, The",1992,Action|Romance|War|Western
36,1597,3.5,Conspiracy Theory,1997,Drama|Mystery|Romance|Thriller
36,58293,3.5,"10,000 BC",2008,Adventure|Romance|Thriller
36,1088,3.0,Dirty Dancing,1987,Drama|Musical|Romance
36,163,3.0,Desperado,1995,Action|Romance|Western


In [88]:
def get_user_profiles():
    user_profiles = {}
    
    ## Create an index_df by joining the interactions_full_df with the articles_df, and get the columns from the interactions_full_df
    #interactions_indexed_df = interactions_full_df[interactions_articles_df['contentId'].isin(articles_df['contentId'])].set_index('personId')

    users_movies_indexed_df = interactions_movies_df.set_index('userId')
    
    ## For every user in the index_df, get all of their contentIds, and get their corresponding tfdifmatrix 

    for userid in users_movies_indexed_df.index.unique():
        interactions_person_df = users_movies_indexed_df.loc[userid]    
        user_item_strengths = np.array(interactions_person_df['rating']).reshape(-1,1) ## n rows x 1 col
    
        itemprofilelist = [tfidf_matrix[item_ids.index(c)] for c in pd.Series(interactions_person_df['movieId'])]
        item_profiles = scipy.sparse.vstack(itemprofilelist)
    
        #Weighted average of item profiles by the interactions strength
        user_item_strengths_weighted_avg = np.sum(item_profiles.multiply(user_item_strengths), axis=0) / np.sum(user_item_strengths)
        user_profile_norm = sklearn.preprocessing.normalize(user_item_strengths_weighted_avg)    
        
        user_profiles[userid] = user_profile_norm
    return user_profiles

In [89]:
user_profiles = get_user_profiles()

In [90]:
len(user_profiles)

156

In [91]:
## Let's take a look in the profile. It is a unit vector of length 90 (length of tf-idf matrix). 
#The value in each position represents how relevant is a token (unigram or bigram).

In [92]:
myprofile = user_profiles[131]
print(myprofile.shape)

(1, 90)


In [94]:
#myprofile.flatten().tolist()

In [95]:
token_relevance = pd.DataFrame(sorted(zip(tfidf_feature_names,user_profiles[131].flatten().tolist())), columns=['token', 'relevance']) 

In [97]:
token_relevance.head()

,token,relevance
0,action,0.233675
1,action adventure,0.128769
2,action animation,0.001998
3,action comedy,0.062557
4,action crime,0.042019


In [98]:
## Sort by highest order of relevance

token_relevance = token_relevance.sort_values(by='relevance', ascending=False)

In [100]:
token_relevance.head(5)

,token,relevance
24,comedy,0.610621
42,drama,0.427634
79,romance,0.301685
0,action,0.233675
31,comedy romance,0.207872


In [101]:
#Compute the cosine similarity between the user profile and all item profiles
person_id = 131
cosine_similarities = cosine_similarity(user_profiles[person_id], tfidf_matrix)
cosine_similarities 

array([[ 0.306343  ,  0.14847164,  0.54569133, ...,  0.18626201,
         0.        ,  0.11740893]])

In [102]:
cosine_similarities.shape

(1, 27278)

In [103]:
#Gets the top similar items
topn = 15
similar_indices = cosine_similarities.argsort().flatten()[-topn:]
similar_indices

array([17284, 12342,  7744, 25706, 12331, 19580,   193, 11242, 19581,
       22637, 19593,  3220, 22632,  7746,  7254], dtype=int64)

In [104]:
#Sort the similar items by similarity
similar_items = sorted([(item_ids[i], cosine_similarities[0,i]) for i in similar_indices], key=lambda x: -x[1])
similar_items

[(87234, 0.62179772859215721),
 (56941, 0.62179772859215721),
 (8294, 0.62179772859215721),
 (121495, 0.62179772859215721),
 (56837, 0.62179772859215721),
 (96943, 0.62179772859215721),
 (195, 0.62179772859215721),
 (47561, 0.62179772859215721),
 (96945, 0.62179772859215721),
 (108473, 0.62179772859215721),
 (96975, 0.62179772859215721),
 (3307, 0.62179772859215721),
 (108427, 0.62179772859215721),
 (8302, 0.62179772859215721),
 (7366, 0.62179772859215721)]

In [105]:
items_to_ignore = interactions_movies_df[interactions_movies_df['userId'] == person_id]['movieId'].tolist()
#items_to_ignore

In [106]:
similar_items_filtered = list(filter(lambda x: x[0] not in items_to_ignore, similar_items)) ##x[0] is the contentId, x[1] is cosine similarity
len(similar_items_filtered)

15

In [107]:
similar_items_filtered

[(87234, 0.62179772859215721),
 (56941, 0.62179772859215721),
 (8294, 0.62179772859215721),
 (121495, 0.62179772859215721),
 (56837, 0.62179772859215721),
 (96943, 0.62179772859215721),
 (195, 0.62179772859215721),
 (47561, 0.62179772859215721),
 (96945, 0.62179772859215721),
 (108473, 0.62179772859215721),
 (96975, 0.62179772859215721),
 (3307, 0.62179772859215721),
 (108427, 0.62179772859215721),
 (8302, 0.62179772859215721),
 (7366, 0.62179772859215721)]

In [108]:
content_rec_df = pd.DataFrame(similar_items_filtered, columns=['movieId', 'recStrength']).head(10)

In [109]:
content_rec_df

,movieId,recStrength
0,87234,0.621798
1,56941,0.621798
2,8294,0.621798
3,121495,0.621798
4,56837,0.621798
5,96943,0.621798
6,195,0.621798
7,47561,0.621798
8,96945,0.621798
9,108473,0.621798


In [110]:
## Merge movies_df with content_rec_df

content_rec_df_completed = content_rec_df.merge(movies_df[['movieId', 'title', 'genres']], how = 'left', left_on = 'movieId',
               right_on = 'movieId')

In [111]:
## Recommended content for personId: 36
content_rec_df_completed

,movieId,recStrength,title,genres
0,87234,0.621798,Submarine (2010),Comedy|Drama|Romance
1,56941,0.621798,P.S. I Love You (2007),Comedy|Drama|Romance
2,8294,0.621798,"Swan, The (1956)",Comedy|Drama|Romance
3,121495,0.621798,Garbage Prince (2011),Comedy|Drama|Romance
4,56837,0.621798,As You Like It (2006),Comedy|Drama|Romance
5,96943,0.621798,"Happy Event, A (Un Heureux Evénement) (2011)",Comedy|Drama|Romance
6,195,0.621798,Something to Talk About (1995),Comedy|Drama|Romance
7,47561,0.621798,Shake It (En kort en lang) (2001),Comedy|Drama|Romance
8,96945,0.621798,Love Lasts Three Years (L'amour dure trois ans...,Comedy|Drama|Romance
9,108473,0.621798,"Laterna, ftoheia kai garyfallo (1958)",Comedy|Drama|Romance


In [219]:
class ContentBasedRecommender:
    
    MODEL_NAME = 'Content-Based'
    
    def __init__(self, movies_df=None):         
        self.movies_df = movies_df        
        
    def get_model_name(self):
        return self.MODEL_NAME
        
    def get_user_profiles(self):
        user_profiles = {}
    
        ## Create an index_df from interactions_movies_df
        users_movies_indexed_df = interactions_movies_df.set_index('userId')
    
        ## For every user in the index_df, get all of their contentIds, and their corresponding tfdifmatrix 
        for userid in users_movies_indexed_df.index.unique():
            interactions_person_df = users_movies_indexed_df.loc[userid]    
            user_item_strengths = np.array(interactions_person_df['rating']).reshape(-1,1) ## n rows x 1 col
    
            itemprofilelist = [tfidf_matrix[item_ids.index(c)] for c in pd.Series(interactions_person_df['movieId'])]
            item_profiles = scipy.sparse.vstack(itemprofilelist)
    
            #Weighted average of item profiles by the interactions strength
            user_item_strengths_weighted_avg = np.sum(item_profiles.multiply(user_item_strengths), axis=0) / np.sum(user_item_strengths)
            user_profile_norm = sklearn.preprocessing.normalize(user_item_strengths_weighted_avg)    
        
            user_profiles[userid] = user_profile_norm
            
        return user_profiles    
        
    def get_similar_items_to_user_profile(self, user_id, topn=1000):
        
        user_profiles = self.get_user_profiles()
        
        #Computes the cosine similarity between the user profile and all item profiles
        cosine_similarities = cosine_similarity(user_profiles[user_id], tfidf_matrix)
        
        #Gets the top similar items
        similar_indices = cosine_similarities.argsort().flatten()[-topn:]
        
        #Sort the similar items by similarity
        similar_items = sorted([(item_ids[i], cosine_similarities[0,i]) for i in similar_indices], key=lambda x: -x[1])
        
        return similar_items
        
    def recommend_items(self, user_id, items_to_ignore=[], topn=10, verbose=False):
        similar_items = self.get_similar_items_to_user_profile(user_id)
        
        #Ignores items the user has already interacted
        similar_items_filtered = list(filter(lambda x: x[0] not in items_to_ignore, similar_items))
        
        rec_df = pd.DataFrame(similar_items_filtered, columns=['movieId', 'recStrength']).head(topn)

       
       # if verbose:
       #     if self.movies_df is None:
       #         raise Exception('"movies_df" is required in verbose mode')

        content_recommendation_df = rec_df.merge(movies_df, how = 'left', 
                                                          left_on = 'movieId', 
                                                          right_on = 'movieId')[['movieId', 'recStrength', 'title_1', 'year', 'genres']]
       
    
        return content_recommendation_df

### Instantiate the ContentBasedRecommender class

In [220]:
cont_rec_model = ContentBasedRecommender(movies_df)

### Call recommend_items for user in train df

In [223]:
users_train = train_df['userId'].tolist()
train_userid = users_train[1]

## movies already watched by train_userid
already_watched_train = train_df[train_df['userId']==train_userid]['movieId'].tolist()

print("Content recommendations for user in train_df: ", train_userid)
cont_rec_model.recommend_items(train_userid, already_watched_train)

Content recommendations for user in train_df:  131


,movieId,recStrength,title_1,year,genres
0,84847,0.621798,Emma,2009,Comedy|Drama|Romance
1,100562,0.621798,"Goddess of 1967, The",2000,Comedy|Drama|Romance
2,84890,0.621798,Incantato (Il cuore altrove),2003,Comedy|Drama|Romance
3,6087,0.621798,If You Could See What I Hear,1982,Comedy|Drama|Romance
4,100203,0.621798,Sundome,2007,Comedy|Drama|Romance
5,70661,0.621798,Tyler Perry's Meet the Browns,2008,Comedy|Drama|Romance
6,4191,0.621798,Alfie,1966,Comedy|Drama|Romance
7,55147,0.621798,27 Missing Kisses,2000,Comedy|Drama|Romance
8,50279,0.621798,My Bollywood Bride,2006,Comedy|Drama|Romance
9,5902,0.621798,Adaptation,2002,Comedy|Drama|Romance


In [225]:
print("Movies already watched by user in train_df: ", train_userid)
train_df[train_df['userId']==train_userid][['movieId', 'year', 'rating', 'title_1', 'genres']].sort_values(by='rating', ascending=False).reset_index(drop=True).head(10)

Movies already watched by user in train_df:  131


,movieId,year,rating,title_1,genres
0,49286,2006,5.0,"Holiday, The",Comedy|Romance
1,2411,1985,5.0,Rocky IV,Action|Drama
2,2409,1979,4.5,Rocky II,Action|Drama
3,6942,2003,4.5,Love Actually,Comedy|Drama|Romance
4,2410,1982,4.5,Rocky III,Action|Drama
5,58998,2008,4.5,Forgetting Sarah Marshall,Comedy|Romance
6,3578,2000,4.5,Gladiator,Action|Adventure|Drama
7,35836,2005,4.0,"40-Year-Old Virgin, The",Comedy|Romance
8,8132,1992,4.0,Gladiator,Action|Drama
9,30825,2004,4.0,Meet the Fockers,Comedy


### Call recommend_items for SAME user in test df

In [227]:
#users_test = test_df['userId'].tolist()
#test_userid = 131 #users_test[1]

## movies already watched by train_userid  in test_df
already_watched_test = test_df[test_df['userId']==train_userid]['movieId'].tolist()

print("Content Recommendation for user in test_df: ", train_userid)
cont_rec_model.recommend_items(train_userid, already_watched_test)

Content Recommendation for user in test_df:  131


,movieId,recStrength,title_1,year,genres
0,84847,0.621798,Emma,2009,Comedy|Drama|Romance
1,100562,0.621798,"Goddess of 1967, The",2000,Comedy|Drama|Romance
2,84890,0.621798,Incantato (Il cuore altrove),2003,Comedy|Drama|Romance
3,6087,0.621798,If You Could See What I Hear,1982,Comedy|Drama|Romance
4,100203,0.621798,Sundome,2007,Comedy|Drama|Romance
5,70661,0.621798,Tyler Perry's Meet the Browns,2008,Comedy|Drama|Romance
6,4191,0.621798,Alfie,1966,Comedy|Drama|Romance
7,55147,0.621798,27 Missing Kisses,2000,Comedy|Drama|Romance
8,50279,0.621798,My Bollywood Bride,2006,Comedy|Drama|Romance
9,5902,0.621798,Adaptation,2002,Comedy|Drama|Romance


In [229]:
print("Movies already watched by user in test_df: ", train_userid)
test_df[test_df['userId']==train_userid][['movieId', 'year', 'rating', 'title_1', 'genres']].sort_values(by='rating', ascending=False).reset_index(drop=True).head(10)

Movies already watched by user in test_df:  131


,movieId,year,rating,title_1,genres
0,58047,2008,4.5,"Definitely, Maybe",Comedy|Drama|Romance
1,66934,2008,4.0,Dr. Horrible's Sing-Along Blog,Comedy|Drama|Musical|Sci-Fi
2,47,1995,3.5,Seven (a.k.a. Se7en),Mystery|Thriller
3,6753,2003,3.5,Secondhand Lions,Children|Comedy|Drama
4,4054,2001,3.5,Save the Last Dance,Drama|Romance
5,442,1993,3.0,Demolition Man,Action|Adventure|Sci-Fi
6,4091,1987,3.0,Can't Buy Me Love,Comedy|Romance
7,2058,1998,3.0,"Negotiator, The",Action|Crime|Drama|Mystery|Thriller
8,1073,1971,3.0,Willy Wonka & the Chocolate Factory,Children|Comedy|Fantasy|Musical
9,2706,1999,3.0,American Pie,Comedy|Romance


### Collaborative Filtering - Matrix Factorization

In [230]:
interactions_movies_df.head()

,userId,movieId,rating,title_1,year,genres
0,1,2,3.5,Jumanji,1995,Adventure|Children|Fantasy
1,1,29,3.5,"City of Lost Children, The (Cité des enfants p...",1995,Adventure|Drama|Fantasy|Mystery|Sci-Fi
2,1,32,3.5,Twelve Monkeys (a.k.a. 12 Monkeys),1995,Mystery|Sci-Fi|Thriller
3,1,47,3.5,Seven (a.k.a. Se7en),1995,Mystery|Thriller
4,1,50,3.5,"Usual Suspects, The",1995,Crime|Mystery|Thriller


In [231]:
#Creating a sparse pivot table with users in rows and movies in columns
users_movies_pivot_matrix_df = interactions_movies_df.pivot(index='userId', 
                                                          columns='movieId', 
                                                          values='rating').fillna(0)

users_movies_pivot_matrix_df.head(10)

movieId,1,2,3,4,5,6,7,8,9,10,...,112623,112852,113453,114180,115617,116797,117511,117590,118696,125916
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,5.0,0.0,3.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,3.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,4.0,0.0,5.0,0.0,0.0,3.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [232]:
users_movies_pivot_matrix_df.shape ## users, movies

(156, 4192)

In [233]:
users_movies_pivot_matrix = users_movies_pivot_matrix_df.as_matrix()
users_movies_pivot_matrix#[:10]

array([[ 0. ,  3.5,  0. , ...,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  4. , ...,  0. ,  0. ,  0. ],
       [ 4. ,  0. ,  0. , ...,  0. ,  0. ,  0. ],
       ..., 
       [ 0. ,  0. ,  0. , ...,  0. ,  0. ,  0. ],
       [ 2.5,  0. ,  0. , ...,  0. ,  0. ,  0. ],
       [ 5. ,  5. ,  2. , ...,  0. ,  0. ,  0. ]])

In [234]:
type(users_movies_pivot_matrix)

numpy.ndarray

In [235]:
users_ids = list(users_movies_pivot_matrix_df.index) ## users are in index
users_ids[:10]

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

### An important decision is the number of factors to factor the user-movies matrix. 
#### The higher the number of factors, the more precise is the factorization in the original matrix reconstructions.
#### Therefore, if the model is allowed to memorize too much details of the original matrix, it may not generalize well for data it was not trained on.

### Reducing the number of factors increases the model generalization.

In [236]:
users_movies_pivot_matrix.shape ## users, movies

(156, 4192)

In [237]:
#The number of factors to factor the user-movies matrix.
NUMBER_OF_FACTORS_MF = 50

In [238]:
#Performs matrix factorization of the original user movies matrix

U, sigma, Vt = svds(users_movies_pivot_matrix, k = NUMBER_OF_FACTORS_MF)

In [239]:
print(U.shape, sigma.shape, Vt.shape)

(156, 50) (50,) (50, 4192)


In [241]:
sigma = np.diag(sigma)
sigma.shape

(50, 50)

#### After the factorization, we try to to reconstruct the original matrix by multiplying its factors. 
#### The resulting matrix is not sparse any more.

### It generates predictions for movies the user has not yet interacted, which we will exploit for recommendations.

In [242]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt)
all_user_predicted_ratings

In [243]:
all_user_predicted_ratings.shape

(156, 4192)

In [246]:
#Converting the reconstructed matrix back to a Pandas dataframe

cf_preds_df = pd.DataFrame(all_user_predicted_ratings, columns = users_movies_pivot_matrix_df.columns, index=users_ids).transpose()
cf_preds_df.head(10)

,1,2,3,4,5,6,7,8,9,10,...,147,148,149,150,151,152,153,154,155,156
movieId,,,,,,,,,,,,,,,,,,,,,
1,0.599611,0.698747,4.453231,0.679727,2.168952,3.296606,-0.310101,1.742570,0.005556,0.681066,...,4.760476,0.853245,1.762139,0.025119,0.418810,0.320853,-0.181354,0.414455,0.748480,4.754336
2,3.262585,0.380625,0.003297,0.289157,0.499934,0.287561,0.495997,1.193663,0.083512,-0.206406,...,-0.029974,-0.491637,0.645078,0.114302,0.161090,1.055905,0.059698,-0.191357,0.265448,4.135406
3,-0.101087,1.030881,-0.312974,0.161806,1.585538,2.117141,3.172050,0.717718,0.023801,0.235253,...,0.076916,-0.110658,0.397496,0.028003,0.148002,-0.176604,-0.015627,0.216320,0.396518,2.150614
4,-0.186174,0.220043,0.069618,-0.216058,-0.381269,0.139844,0.061206,-0.017122,0.007479,-0.082773,...,-0.084542,-0.025495,-0.147291,0.072158,-0.083188,0.058978,0.150028,0.026279,0.242424,2.179800
5,-0.061950,0.364409,-0.320773,0.240251,0.462498,0.827814,-0.022522,0.578909,0.008623,-0.138768,...,0.168842,-0.090138,0.223516,0.031387,0.145750,-0.461182,-0.073557,0.158827,-0.063693,2.002719
6,-0.430448,0.850223,-0.120608,0.441755,0.972962,1.726309,-0.313508,0.959346,-0.059669,0.280324,...,3.839314,0.666961,0.052517,-0.069954,0.146876,-0.227345,0.272619,5.293265,0.844549,4.311804
7,0.117004,0.613558,-0.177138,-0.137557,1.051998,2.391056,3.574820,0.129490,-0.015731,-0.037278,...,-0.005258,3.254430,0.166188,-0.121475,0.119660,-0.671689,-0.351043,-0.253093,0.201336,4.190984
8,-0.003157,0.035250,-0.024831,0.038769,0.094140,0.030632,0.059857,0.146352,0.034943,0.019137,...,-0.005716,0.118346,0.037616,-0.007123,0.024990,-0.020638,0.033221,-0.045760,-0.033029,0.125666
9,0.117493,0.231167,0.032385,0.133503,-0.143718,0.110620,-0.050339,0.663361,-0.050998,0.138656,...,0.029639,-0.242202,0.065592,-0.021912,0.135259,-0.449867,-0.477942,-0.071198,0.077894,3.374176


In [248]:
cf_preds_df.shape ## movies, users

(4192, 156)

In [258]:
class CollabFilterRecommender:
    
    MODEL_NAME = 'Collaborative Filtering'
    
    def __init__(self, cf_predictions_df, items_df=None):
        self.cf_predictions_df = cf_predictions_df
        self.items_df = items_df
        
    def get_model_name(self):
        return self.MODEL_NAME
        
    def recommend_items(self, user_id, items_to_ignore=[], topn=10, verbose=False):
        # Get and sort the user's predictions
        sorted_user_predictions = self.cf_predictions_df[user_id].sort_values(ascending=False).reset_index().rename(columns={user_id: 'recStrength'})

        # Recommend the highest predicted rating movies that the user hasn't seen yet.
        recommendations_df = sorted_user_predictions[~sorted_user_predictions['movieId'].isin(items_to_ignore)] \
                               .sort_values('recStrength', ascending = False).reset_index(drop=True).head(topn)

        #if verbose:
        #    if self.items_df is None:
        #        raise Exception('"items_df" is required in verbose mode')

        recommendations_df = recommendations_df.merge(self.items_df, how = 'left', 
                                                          left_on = 'movieId', 
                                                          right_on = 'movieId')[['movieId', 'recStrength', 'title_1', 'year', 'genres']]


        return recommendations_df

### Instantiate the CollabFilterRecommender model

In [259]:
collab_filter_model = CollabFilterRecommender(cf_preds_df, movies_df)

### Call recommend_items for user in train df

In [262]:
users_train = train_df['userId'].tolist()
train_userid = users_train[1]

## movies already watched by train_userid
already_watched_train = train_df[train_df['userId']==train_userid]['movieId'].tolist()

print("Movie recommendations for user %d in train_df using Collaborative filtering: " %train_userid)
collab_filter_model.recommend_items(train_userid, already_watched_train)

Movie recommendations for user 131 in train_df using Collaborative filtering: 


,movieId,recStrength,title_1,year,genres
0,58047,4.060635,"Definitely, Maybe",2008,Comedy|Drama|Romance
1,66934,3.963107,Dr. Horrible's Sing-Along Blog,2008,Comedy|Drama|Musical|Sci-Fi
2,2706,3.797807,American Pie,1999,Comedy|Romance
3,733,3.485601,"Rock, The",1996,Action|Adventure|Thriller
4,442,3.387927,Demolition Man,1993,Action|Adventure|Sci-Fi
5,4054,3.322113,Save the Last Dance,2001,Drama|Romance
6,6753,3.274944,Secondhand Lions,2003,Children|Comedy|Drama
7,4034,3.129634,Traffic,2000,Crime|Drama|Thriller
8,1517,3.073546,Austin Powers: International Man of Mystery,1997,Action|Adventure|Comedy
9,593,3.009496,"Silence of the Lambs, The",1991,Crime|Horror|Thriller


In [263]:
print("Movies already watched by user in train_df: ", train_userid)
train_df[train_df['userId']==train_userid][['movieId', 'year', 'rating', 'title_1', 'genres']].sort_values(by='rating', ascending=False).head(5)

Movies already watched by user in train_df:  131


,movieId,year,rating,title_1,genres
16082,49286,2006,5.0,"Holiday, The",Comedy|Romance
15873,2411,1985,5.0,Rocky IV,Action|Drama
15871,2409,1979,4.5,Rocky II,Action|Drama
16036,6942,2003,4.5,Love Actually,Comedy|Drama|Romance
15872,2410,1982,4.5,Rocky III,Action|Drama


In [264]:
#users_test = test_df['userId'].tolist()
#test_userid = 131 #users_test[1]

## movies already watched by train_userid
already_watched_test = test_df[test_df['userId']==train_userid]['movieId'].tolist()


print("Movie recommendations for user %d in test_df using Collaborative filtering: " %train_userid)
collab_filter_model.recommend_items(train_userid, already_watched_test)

Movie recommendations for user 131 in test_df using Collaborative filtering: 


,movieId,recStrength,title_1,year,genres
0,2411,4.970993,Rocky IV,1985,Action|Drama
1,3578,4.908700,Gladiator,2000,Action|Adventure|Drama
2,2410,4.529725,Rocky III,1982,Action|Drama
3,49286,4.512483,"Holiday, The",2006,Comedy|Romance
4,2409,4.504337,Rocky II,1979,Action|Drama
5,58998,4.244268,Forgetting Sarah Marshall,2008,Comedy|Romance
6,35836,4.052606,"40-Year-Old Virgin, The",2005,Comedy|Romance
7,2959,4.030157,Fight Club,1999,Action|Crime|Drama|Thriller
8,261,3.977819,Little Women,1994,Drama
9,5952,3.972308,"Lord of the Rings: The Two Towers, The",2002,Adventure|Fantasy


In [265]:
print("Movies already watched by user in test_df: ", train_userid)
test_df[test_df['userId']==train_userid][['movieId', 'year', 'rating', 'title_1', 'genres']].sort_values(by='rating', ascending=False).head(5)

Movies already watched by user in test_df:  131


,movieId,year,rating,title_1,genres
16098,58047,2008,4.5,"Definitely, Maybe",Comedy|Drama|Romance
16123,66934,2008,4.0,Dr. Horrible's Sing-Along Blog,Comedy|Drama|Musical|Sci-Fi
15710,47,1995,3.5,Seven (a.k.a. Se7en),Mystery|Thriller
16034,6753,2003,3.5,Secondhand Lions,Children|Comedy|Drama
15963,4054,2001,3.5,Save the Last Dance,Drama|Romance


### Correlation between movies

movieId,1,2,3,4,5,6,7,8,9,10,...,112623,112852,113453,114180,115617,116797,117511,117590,118696,125916
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,5.0,0.0,3.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,3.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,4.0,0.0,5.0,0.0,0.0,3.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [269]:
movie_id = 1

movie_watched = users_movies_pivot_df[movie_id]
movie_watched.head()

userId
1    0.0
2    0.0
3    4.0
4    0.0
5    0.0
Name: 1, dtype: float64

In [270]:
similarity_with_other_movies = users_movies_pivot_matrix_df_corr.corrwith(movie_watched)  # find correlation between movieid 1 and other movies
similarity_with_other_movies.head(10)

movieId
1     1.000000
2     0.110350
3     0.257602
4     0.032240
5     0.180903
6     0.361697
7     0.232298
8    -0.035042
9     0.064014
10    0.199418
dtype: float64

In [275]:
similarity_df = pd.DataFrame(data=similarity_with_other_movies, columns=['Correlation']).reset_index()#index=users_movies_pivot_matrix_df_corr.columns
similarity_df.head(10)

,movieId,Correlation
0,1,1.000000
1,2,0.110350
2,3,0.257602
3,4,0.032240
4,5,0.180903
5,6,0.361697
6,7,0.232298
7,8,-0.035042
8,9,0.064014
9,10,0.199418


In [276]:
similarity_df.sort_values(by='Correlation', inplace=True, ascending=False)
similarity_df.head()

,movieId,Correlation
0,1,1.000000
2039,3114,0.464813
568,802,0.448421
30,32,0.424000
82,95,0.385274


In [277]:
sim_df = similarity_df.merge(movies_df[['movieId', 'title_1', 'year', 'genres']], how = 'left', left_on = 'movieId',
               right_on = 'movieId')
sim_df.head()

,movieId,Correlation,title_1,year,genres
0,1,1.000000,Toy Story,1995,Adventure|Animation|Children|Comedy|Fantasy
1,3114,0.464813,Toy Story 2,1999,Adventure|Animation|Children|Comedy|Fantasy
2,802,0.448421,Phenomenon,1996,Drama|Romance
3,32,0.424000,Twelve Monkeys (a.k.a. 12 Monkeys),1995,Mystery|Sci-Fi|Thriller
4,95,0.385274,Broken Arrow,1996,Action|Adventure|Thriller


In [ ]:
# Recommend the highest predicted rating movies that the user hasn't seen yet.
user_id = 36
## get list of already interacted content ids (in order to avoid coming up in the recommendation list)
collab_ignore_items_1 = interactions_movies_df[interactions_movies_df['userId'] == user_id]['movieId'].tolist()
#collab_ignore_items


In [ ]:
collab_recommendations_df_1 = sim_df[~sim_df['movieId'].isin(collab_ignore_items_1)] \
                               .sort_values('Correlation', ascending = False) \
                               .head(10)

In [ ]:
collab_recommendations_df_1

In [375]:
interactions_movies_df_new = interactions_movies_df[:500]

In [376]:
interactions_movies_df_new['movieId'].nunique()

415

In [431]:
#Creating a sparse pivot table with users in rows and items in columns
users_movies_pivot_df = interactions_movies_df_new.pivot(index='userId', 
                                                          columns='movieId', 
                                                          values='rating').fillna(0)

users_movies_pivot_df.head(10)

movieId,1,2,3,6,10,11,17,19,24,29,...,7454,7482,7757,8368,8482,8507,8636,8690,8961,31696
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,...,4.0,3.0,4.0,4.0,3.5,5.0,4.5,3.5,4.0,4.0
2,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,3.0,4.0,0.0,0.0,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,3.0,0.0,0.0,0.0,5.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [464]:
class ItemsCorrelation:
    
    MODEL_NAME = 'Correlation'
    
    def __init__(self, pivot_df, items_df=None):
        self.pivot_df = pivot_df
        self.items_df = items_df
        
        
    def get_model_name(self):
        return self.MODEL_NAME
    
    
    def corr_with_movies(mov_id):
        movies_watched = self.pivot_df[mov_id]
        correlation_with_other_movies = self.pivot_df.corrwith(movie_watched)            
        correlation_df = pd.DataFrame(data=correlation_with_other_movies, columns=['Correlation'], index=self.pivot_df.columns)#.reset_index()    
        correlation_df['Correlation_with_Movie'] = mov_id
        correlation_df = correlation_df.sort_values(by='Correlation', ascending=False)
        correlation_df = correlation_df.reset_index()         
        return correlation_df
        
    
    def create_correlation_df(self):
        corr_df = pd.DataFrame()
        for movieid in self.pivot_df.columns:
            data = corr_with_movies(movieid)    
            corr_df = corr_df.append(data)        
        return corr_df  
 

    def recommend_items(self, user_id, items_to_ignore=[], topn=10, verbose=False):
        corr_df = self.create_correlation_df()
        recommendations_df = corr_df.merge(self.items_df, how = 'left', 
                                                          left_on = 'movieId', 
                                                          right_on = 'movieId')[['movieId', 'Correlation_with_Movie', 'title_1', 'year', 'genres']]


        return recommendations_df
 

### Instantiate the ItemCorrelation class

In [465]:
item_corr_model = ItemsCorrelation(users_movies_pivot_df, movies_df)

### Call recommend_items for user in train_df

In [466]:
users_train = train_df['userId'].tolist()
train_userid = users_train[1]

## movies already watched by train_userid
already_watched_train = train_df[train_df['userId']==train_userid]['movieId'].tolist()

print("Movie recommendations for user %d in train_df using Correlation: " %train_userid)
item_corr_model.recommend_items(train_userid, already_watched_train)

Movie recommendations for user 131 in train_df using Correlation: 


,movieId,Correlation_with_Movie,title_1,year,genres
0,1,1,Toy Story,1995,Adventure|Animation|Children|Comedy|Fantasy
1,1603,1,Mimic,1997,Horror|Sci-Fi|Thriller
2,1307,1,When Harry Met Sally...,1989,Comedy|Romance
3,1330,1,April Fool's Day,1986,Horror
4,1345,1,Carrie,1976,Drama|Fantasy|Horror|Thriller
5,1372,1,Star Trek VI: The Undiscovered Country,1991,Action|Mystery|Sci-Fi
6,1373,1,Star Trek V: The Final Frontier,1989,Action|Sci-Fi
7,1376,1,Star Trek IV: The Voyage Home,1986,Adventure|Comedy|Sci-Fi
8,2949,1,Dr. No,1962,Action|Adventure|Thriller
9,1396,1,Sneakers,1992,Action|Comedy|Crime|Drama|Sci-Fi


In [463]:
print("Movies already watched by user in train_df: ", train_userid)
train_df[train_df['userId']==train_userid][['movieId', 'year', 'rating', 'title_1', 'genres']].sort_values(by='rating', ascending=False).head(5)

Movies already watched by user in train_df:  131


,movieId,year,rating,title_1,genres
16082,49286,2006,5.0,"Holiday, The",Comedy|Romance
15873,2411,1985,5.0,Rocky IV,Action|Drama
15871,2409,1979,4.5,Rocky II,Action|Drama
16036,6942,2003,4.5,Love Actually,Comedy|Drama|Romance
15872,2410,1982,4.5,Rocky III,Action|Drama
